In [3]:
%load_ext autoreload
%autoreload 2

In [11]:
import pandas as pd 
from pathlib import Path
import toml

from bioblp.data import COL_SOURCE, COL_TARGET,COL_EDGE
from bioblp.data import create_random_splits

In [ ]:
DATA_DIR = Path("../data")
SHARED_DATA_DIR = Path("/home/jovyan/workbench-shared-folder/bioblp/data")
config_path = DATA_DIR.joinpath("conf/complex-biokg-20220826.toml")
biokg_mini_path = SHARED_DATA_DIR.joinpath("raw/biokg.links_sample.tsv")
biokg_path = SHARED_DATA_DIR.joinpath("raw/biokg.links.tsv")

## Hetionet

In [21]:
from pykeen.datasets import Hetionet
from pykeen.datasets import get_dataset

ds = get_dataset(dataset=Hetionet)
ds.summarize()

EagerDataset (create_inverse_triples=False)
Name        Entities    Relations      Triples
----------  ----------  -----------  ---------
Training    45158       24             1800157
Testing     45158       24              225020
Validation  45158       24              225020
Total       -           -              2250197
Head                     Relation    tail
-----------------------  ----------  ------------
Anatomy::UBERON:0000002  AdG         Gene::10005
Anatomy::UBERON:0000002  AdG         Gene::114804
Anatomy::UBERON:0000002  AdG         Gene::118670
Anatomy::UBERON:0000002  AdG         Gene::128989
Anatomy::UBERON:0000002  AdG         Gene::132851



In [39]:
triples = Hetionet().factory_dict
test = pd.DataFrame(triples['testing'].triples, columns=[[COL_SOURCE, COL_EDGE, COL_TARGET]])
train = pd.DataFrame(triples['training'].triples, columns=[[COL_SOURCE, COL_EDGE, COL_TARGET]])
valid = pd.DataFrame(triples['validation'].triples, columns=[[COL_SOURCE, COL_EDGE, COL_TARGET]])

Reconstructing all label-based triples. This is expensive and rarely needed.
Reconstructing all label-based triples. This is expensive and rarely needed.
Reconstructing all label-based triples. This is expensive and rarely needed.


In [41]:
len(test)/(len(train)+ len(test) +len(valid))

0.10000013332166029

In [42]:
SAVE_SPLITS_TO_DISK = False
hetio_dataset_name = 'hetionet_random_801010'
hetio_datasplits_dir =  DATA_DIR.joinpath("raw/hetionet_splits")

if SAVE_SPLITS_TO_DISK:
    save_splits(train_df=train,
               test_df=test, 
               valid_df=valid,
               dataset_name=hetio_dataset_name",
               out_dir=hetio_datasplits_dir)

saved to ../data/raw/hetionet_splits


In [15]:
def load_toml(toml_path: str) -> dict:
    toml_path = Path(toml_path)

    config = {}

    with open(toml_path, "r") as f:
        config = toml.load(f)

    return config

config = load_toml(config_path)
config

{'train_triples': 'data',
 'valid_triples': 'data',
 'test_triples': 'data',
 'model': 'complex',
 'dimension': 256,
 'loss_fn': 'crossentropy',
 'loss_margin': 1.0,
 'optimizer': 'adagrad',
 'learning_rate': 0.01,
 'regularizer': 1e-06,
 'num_epochs': 100,
 'batch_size': 1024,
 'eval_batch_size': 16,
 'num_negatives': 512,
 'add_inverses': False,
 'early_stopper': 'both.realistic.inverse_harmonic_mean_rank',
 'search_train_batch_size': False,
 'search_eval_batch_size': False,
 'log_wandb': False}

### Training

```bash
$ python -m bioblp.train_argparse --conf /home/jovyan/BioBLP/data/conf/complex-hetionet-20220826.toml
```